In [1]:
from PIL import Image
import torch
import torch.nn as nn
import torchvision
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import time
from torch.utils.tensorboard import SummaryWriter

Creating a dataframe with image url and label

In [2]:
root_directory = os.getcwd()

In [3]:
paper_url = root_directory + '\\paper'
rock_url = root_directory + '\\rock'
scissors_url = root_directory + '\\scissors'

In [4]:
os.chdir(paper_url)

In [5]:
value_dict = {0:'paper',1:'rock',2:'scissors'}

In [6]:
paper_images_names = os.listdir()

In [7]:
df = pd.DataFrame(paper_images_names, np.zeros((len(paper_images_names))))

In [8]:
os.chdir(rock_url)
rock_images_names = os.listdir()
df = df.append(pd.DataFrame(rock_images_names , np.ones((len(rock_images_names)))))

In [9]:
os.chdir(scissors_url)
scissors_images_names = os.listdir()
df = df.append(pd.DataFrame(scissors_images_names , np.ones((len(scissors_images_names))) * 2))

In [10]:
df['labels'] = df.index

In [11]:
df['labels'] = df['labels'].astype(int)

In [12]:
df.columns = ['images' , 'labels']

Importing the data

In [13]:
class RockPaperScissors(Dataset):
    def __init__(self,directory,df,transform = None):
        self.transform = transform
        self.df = df
        self.directory = directory
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,index):
        img_url = os.path.join(self.directory , 'everything' ,self.df.iloc[index,0])
        img = np.asarray(Image.open(img_url))
        label = torch.tensor(int(self.df.iloc[index,1]))
        
        if self.transform:
            img = self.transform(img)
        return img.to(device) , label.to(device)
        

In [14]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [15]:
dataset = RockPaperScissors(root_directory,df,transform)

train_split , test_split = torch.utils.data.random_split(dataset,[1458,730])

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [17]:
train_loader = DataLoader(dataset = train_split , batch_size = 8 , shuffle=True)
test_loader = DataLoader(dataset = test_split , batch_size = 8 , shuffle=True)

Using mobilenet pretrained model

In [18]:
# lr = 1e-3
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.classifier[1].parameters(),lr=lr)

Training

In [19]:
learning_rates = [0.01]
os.chdir(root_directory)
start = time.perf_counter()
num_epochs = 30

for lr in learning_rates:
    step = 0
    model = torchvision.models.mobilenet_v2(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    model.classifier[1] = nn.Linear(in_features=1280 , out_features = 3 , bias=True)
    model = model.to(device)
    model.train()
    optimizer = torch.optim.Adam(model.classifier[1].parameters(),lr = lr)

    writer = SummaryWriter(f'runs/Testing/Learning rate = {lr}')
    for epoch in range(num_epochs):
        for i, (x,y) in enumerate(train_loader):
            #Training the model
            x,y = x.to(device) , y.to(device)
            y_hat = model(x)
            loss = criterion(y_hat,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            #Training Accuracy
            _ , pred = y_hat.max(1)
            num_correct = (pred == y).sum()
            running_train_acc = float(num_correct)/float(x.shape[0])

            #TensorBoard
            writer.add_scalar('Training Loss', loss, global_step = step)
            writer.add_scalar('Training Accuracy' , running_train_acc , global_step = step)
            step += 1
end = time.perf_counter()

In [20]:
print(f'Training time = {end-start}')

Training time = 1560.6419165


Testing

In [21]:
correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for x,y in test_loader:
            x , y = x.to(device) , y.to(device)
            y_hat = model(x)
            _ , pred = y_hat.max(1)
            correct += (pred == y).sum()
            total += pred.shape[0]
    print(f'model score = {correct/total}')
    model.train()

model score = 0.9945205450057983
